## 기본설정 및 함수정의

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.graph_objs as go
import plotly.offline as offline
from folium.plugins import HeatMapWithTime
from plotly.subplots import make_subplots
import folium
from folium import plugins
from folium.plugins import HeatMap
from folium import FeatureGroup
import json
import math
import re
from datetime import datetime
import os
import glob
import subprocess
from bs4 import BeautifulSoup as bs
from shapely.geometry import Point, Polygon, LineString
from shapely.ops import unary_union
import geopandas as gpd
from geopandas import GeoSeries
import pyproj
from tqdm import tqdm
from keplergl import KeplerGl

# 모든 열이 생략되지 않도록 설정
pd.set_option('display.max_columns', None)

# tqdm의 pandas 적용
tqdm.pandas()

# 아산시청 위도, 경도
Asan = [36.789882248764656, 127.00274491353838]

# Polygon을 만드는 함수
def make_pol(x):
    try:
        return Polygon(x[0])
    except:
        return Polygon(x[0][0])
    
# Linestring을 만드는 함수
def make_lin(x):
    try:
        return LineString(x)
    except:
        return LineString(x[0])

# 데이터프레임을 GeoPandas 데이터프레임으로 변환하는 함수 정의
def geo_transform(DataFrame) :
    # csv to geopandas
    # lon, lat data를 geometry로 변경
    DataFrame['lat'] = DataFrame['lat'].astype(float)
    DataFrame['lon'] = DataFrame['lon'].astype(float)
    DataFrame['geometry'] = DataFrame.apply(lambda row : Point([row['lon'], row['lat']]), axis=1) # 위도 및 경도롤 GeoPandas Point 객체로 변환
    DataFrame = gpd.GeoDataFrame(DataFrame, geometry='geometry')
    DataFrame.crs = {'init':'epsg:4326'} # geopandas 데이터프레임의 좌표계를 EPSG 4326으로 설정
    DataFrame = DataFrame.to_crs({'init':'epsg:4326'}) # 데이터프레임의 좌표계를 자체 좌표계에서 EPSG 4326으로 변환
    return DataFrame

c:\Users\HJK\anaconda3\envs\GIS\lib\site-packages\geopandas\_compat.py:124: UserWarning:

The Shapely GEOS version (3.11.2-CAPI-1.17.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.

C:\Users\HJK\AppData\Local\Temp\ipykernel_17420\3880737364.py:23: DeprecationWarning:

Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch should be smooth. However, if you are using

#### 아산시 행정경계 (출처 - 통계지리정보서비스 2023년 센서스용 행정구역경계(읍면동))

In [2]:
# shp to geodataframe convert
shapefile_path = "SBJ_2405_001/_census_data_2023_bnd_dong_bnd_dong_34040_2023_2023/bnd_dong_34040_2023_2023_2Q.shp"
asan_gdf = gpd.read_file(shapefile_path)
asan_gdf = asan_gdf.to_crs(epsg=4326) #EPSG4326 형식으로 변환

# 행정동 구분에 따라 색 구분하는 함수
def hjd_color(name):
    if name[-1] == '읍':
        return 'green'
    elif name[-1] == '면':
        return 'yellow'
    elif name[-1] == '동':
        return 'red'

asan_gdf['color'] = asan_gdf['ADM_NM'].apply(hjd_color)

#### 격자(매핑용)

In [3]:
# GeoJSON 파일 불러오기
with open('SBJ_2405_001/12.아산시_격자(매핑용).geojson', encoding="UTF8") as geojson_file:
    geojson_data = json.load(geojson_file)
grid_map_df = pd.json_normalize(geojson_data['features'])
grid_map_df['geometry'] = grid_map_df['geometry.coordinates'].apply(lambda x : make_pol(x))
# grid_map_df 데이터프레임을 GeoDataFrame으로 변환
grid_map_df = gpd.GeoDataFrame(grid_map_df, geometry='geometry')

#### 아산시 상세 도로망

In [4]:
# GeoJSON 파일 불러오기
with open('SBJ_2405_001/4.아산시_상세도로망.geojson', encoding="UTF8") as geojson_file:
    geojson_data = json.load(geojson_file)
roadsystem_df = pd.json_normalize(geojson_data['features'])
roadsystem_df['geometry'] = roadsystem_df['geometry.coordinates'].apply(lambda x : make_lin(x))
roadsystem_df['properties.link_id'] = roadsystem_df['properties.link_id'].astype(str)
# roadsystem_df 데이터프레임을 GeoDataFrame으로 변환
roadsystem_df = gpd.GeoDataFrame(roadsystem_df, geometry='geometry')

#### 아산시 병원진료과목정보

In [5]:
# 병원 데이터프레임을
treatment_df = pd.read_csv('SBJ_2405_001/22.아산시_병원진료과목정보.csv')

In [6]:
# 병원별 진료과목 list mapping한 dict생성
hos_dic = {}
for hos in tqdm(treatment_df['mdcl_inst'].unique()):
    subset = treatment_df[treatment_df['mdcl_inst'] == hos]
    hos_dic[hos] = subset['mdcl_mjr_nm'].tolist()

100%|██████████| 331/331 [00:00<00:00, 3351.98it/s]


#### 아산시 병원정보

In [7]:
# 병원 데이터프레임을 GeoPandas 데이터프레임으로 변환
hospital_df = pd.read_csv('SBJ_2405_001/21.아산시_병원정보.csv')
hospital_df = geo_transform(hospital_df)
# 병원별 진료과목 mapping
hospital_df['mjr'] = hospital_df['mdcl_inst'].map(hos_dic)
# Update 'mjr' column where it is null with 'mdcl_gbn' column value as a list
hospital_df.loc[hospital_df['mjr'].isnull(), 'mjr'] = hospital_df['mdcl_gbn'].apply(lambda x: [x])

c:\Users\HJK\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

c:\Users\HJK\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6



#### 아산시 약국현황

In [8]:
# 약국 데이터프레임을 GeoPandas 데이터프레임으로 변환
pharmacy_df = pd.read_csv('SBJ_2405_001/23.아산시_약국현황.csv')
pharmacy_df = geo_transform(pharmacy_df)

c:\Users\HJK\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

c:\Users\HJK\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6



#### 아산시 AED위치정보

In [9]:
AED_df = pd.read_csv('SBJ_2405_001/8.아산시_자동심장충격기(AED)위치정보.csv')
AED_df = geo_transform(AED_df)

c:\Users\HJK\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

c:\Users\HJK\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6



#### 아산시 119 안전센터

In [10]:
df_119 = pd.read_csv('SBJ_2405_001/24.아산시_119안전센터정보.csv')
df_119 = geo_transform(df_119)

c:\Users\HJK\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

c:\Users\HJK\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6



#### 아산시 질병분류별 진료현황

In [11]:
# 질병분류별 진료현황 데이터프레임
present_condition_df = pd.read_csv('SBJ_2405_001/20.아산시_질병중분류별진료현황.csv') # engine='python'
# 이상하게 연도별 데이터값이 같기 때문에 연도분할 필요없이 한 연도로 활용해야겠다 판단(데이터 미스)
present_condition_df = present_condition_df[present_condition_df['clnc_yr'] == 2020]
present_condition_df.drop(columns=['clnc_yr', 'clnc_cnt', 'clnc_exp'], inplace=True)
# 질병코드 : 질병명 딕셔너리 생성
dss_dic = dict(zip(present_condition_df['dss_gbn'], present_condition_df['dss_nm']))
present_condition_df_in = present_condition_df[present_condition_df['inout'] == '관내']
present_condition_df_out = present_condition_df[present_condition_df['inout'] == '관외']

In [ ]:
# 데이터프레임의 row 수 계산
row_counts = {
    'Dataset': ['아산시 관내', '아산시 관외'],
    'Row Count': [len(present_condition_df_in), len(present_condition_df_out)]
}

row_counts_df = pd.DataFrame(row_counts)

# 원형 그래프 그리기
fig = px.pie(row_counts_df, 
            values='Row Count', 
            names='Dataset')

# 그래프 크기 조절 및 annotation 추가
fig.update_layout(
    width=800, 
    height=600, 
)

# 그래프 보여주기
fig.show()

In [ ]:
# clnc_age 컬럼의 고유값 순서
unique_ages = present_condition_df_in['clnc_age'].unique()

# clnc_age 컬럼에 대한 value_counts 계산
in_age_counts = present_condition_df_in['clnc_age'].value_counts().reindex(unique_ages).fillna(0)
out_age_counts = present_condition_df_out['clnc_age'].value_counts().reindex(unique_ages).fillna(0)

# 데이터프레임으로 변환
in_age_df = in_age_counts.reset_index()
in_age_df.columns = ['clnc_age', 'count']

out_age_df = out_age_counts.reset_index()
out_age_df.columns = ['clnc_age', 'count']

# in_age_df 시각화
fig_in_age = px.bar(in_age_df, x='clnc_age', y='count', 
                    labels={'clnc_age': 'Age', 'count': 'Count'}, 
                    color='count', color_continuous_scale=px.colors.sequential.Plasma)

# out_age_df 시각화
fig_out_age = px.bar(out_age_df, x='clnc_age', y='count', 
                     labels={'clnc_age': 'Age', 'count': 'Count'}, 
                     color='count', color_continuous_scale=px.colors.sequential.Plasma)

# 그래프 크기 조절 및 보여주기
fig_in_age.update_layout(width=800, height=600)
fig_out_age.update_layout(width=800, height=600)

fig_in_age.show()
fig_out_age.show()

In [ ]:
# 'mdcl_inst_sido' 컬럼 값의 비율 계산
value_counts = pd.concat([present_condition_df_out['mdcl_inst_sido'], present_condition_df_in['mdcl_inst_sgg']]).value_counts(normalize=True) * 100

# 데이터프레임 생성
value_counts_df = value_counts.reset_index()
value_counts_df.columns = ['mdcl_inst_sido', 'proportion']
# 값을 높은 순에서 낮은 순으로 정렬
value_counts_df = value_counts_df.sort_values(by='proportion', ascending=False)

# 막대 그래프 그리기
fig = px.bar(value_counts_df, 
            x='mdcl_inst_sido', 
            y='proportion', 
            labels={'proportion': '비율(%)', 'mdcl_inst_sido': '행정구역'},
            color='proportion', 
            color_continuous_scale='plasma_r',
            text='proportion')

# 그래프 크기 조절
fig.update_layout(width=1300, height=800)
fig.update_traces(texttemplate='%{text:.2f}%', textposition='outside')
# 그래프 보여주기
fig.show()

##### 관내/외 진료기록 비교

In [12]:
out_dic = present_condition_df_out['dss_nm'].value_counts().to_dict()
in_dic = present_condition_df_in['dss_nm'].value_counts().to_dict()

# 한쪽에만 있는 key값과 그 value로만 이루어진 dict 생성
out_keys = set(out_dic.keys())
in_keys = set(in_dic.keys())

out_only_keys = out_keys - in_keys
in_only_keys = in_keys - out_keys
common_keys = out_keys & in_keys

# 아산시 관외에만 있는 진료기록을 포함하는 dict
out_only_dict = {key: out_dic[key] for key in out_only_keys}

# 아산시 관내에만 있는 진료기록을 포함하는 dict
in_only_dict = {key: in_dic[key] for key in in_only_keys}

# 공통 key값에 대해 큰 value에서 작은 value를 뺀 value가 매핑된 dict 생성
diff_dict = {key: out_dic[key] - in_dic[key] for key in common_keys}

In [ ]:
# 각 딕셔너리를 데이터프레임으로 변환 및 내림차순 정렬
out_only_df = pd.DataFrame(list(out_only_dict.items()), columns=['key', 'value']).sort_values(by='value', ascending=False)
in_only_df = pd.DataFrame(list(in_only_dict.items()), columns=['key', 'value']).sort_values(by='value', ascending=False)
diff_df = pd.DataFrame(list(diff_dict.items()), columns=['key', 'value']).sort_values(by='value', ascending=False)

# out_only_dict 시각화
fig_out_only = px.bar(out_only_df, x='key', y='value', 
                      labels={'key': '진료항목', 'value': '진료건수'}, 
                      color='value', color_continuous_scale=px.colors.sequential.Plasma_r)

# in_only_dict 시각화
fig_in_only = px.bar(in_only_df, x='key', y='value', 
                     labels={'key': '진료항목', 'value': '진료건수'}, 
                     color='value', color_continuous_scale=px.colors.sequential.Plasma_r)

# diff_dict 시각화
fig_diff = px.bar(diff_df, x='key', y='value', 
                  labels={'key': '진료항목', 'value': '진료건수 격차 (외부 - 내부)'}, 
                  color='value', color_continuous_scale=px.colors.sequential.Plasma_r)

# 그래프 크기 조절 및 보여주기
fig_out_only.update_layout(width=800, height=600)
fig_in_only.update_layout(width=800, height=600)
fig_diff.update_layout(width=1500, height=1000)

fig_out_only.show()
fig_in_only.show()
fig_diff.show()

#### 아산시 이송 arc맵(keplerGL)

In [13]:
city_loc_dic = {'서울특별시': [37.55545268653409, 126.97181180156029],
                '부산광역시': [35.115225, 129.042243],
                '대구광역시': [35.876083, 128.596031],
                '인천광역시': [37.456259834754206, 126.70519351959229],
                '광주광역시': [35.16535375285658, 126.90921306610107],
                '대전광역시': [36.332326, 127.434211],
                '울산광역시': [35.551431, 129.138506],
                '세종특별자치시': [36.4799919, 127.2890511],
                '경기도': [37.2893525, 127.0535312],
                '강원특별자치도': [37.8853984, 127.7297758],
                '충청북도': [36.647091, 127.392349],
                '충청남도': [36.659158680020006, 126.67294263839722],
                '전북특별자치도': [35.82037086432629, 127.10876941680908],
                '전라남도': [34.81623442720627, 126.46291494369507],
                '경상북도': [36.576024890131045, 128.50557804107666],
                '경상남도': [35.237837987833686, 128.69189500808716],
                '제주특별자치도': [33.4996213, 126.5311884]}

# 굵기 표현을위해 도시별 비율 dict 생성
city_rate_dic = dict(present_condition_df_out['mdcl_inst_sido'].value_counts(normalize=True) * 100)

# 시작점과 끝점 좌표로 데이터프레임 생성
arc_data = {
    'start_lat': [Asan[0]] * len(city_loc_dic),
    'start_lon': [Asan[1]] * len(city_loc_dic),
    'end_lat': [p[0] for p in city_loc_dic.values()],
    'end_lon': [p[1] for p in city_loc_dic.values()],
    'name': city_loc_dic.keys(),
    'proportion': [city_rate_dic[city] for city in city_loc_dic.keys()]
}

arc_df = pd.DataFrame(arc_data)

In [ ]:
# KeplerGl 지도 객체 생성
map_1 = KeplerGl(height=600)

# 데이터 추가
map_1.add_data(data=arc_df, name="Arc Data")

# 아크 레이어 구성
config = {
    'version': 'v1',
    'config': {
        'visState': {
            'filters': [],
            'layers': [
                {
                    'id': 'arc_layer',
                    'type': 'arc',
                    'config': {
                        'dataId': 'Arc Data',
                        'label': 'Arcs',
                        'color': [30, 150, 190],
                        'columns': {
                            'lat0': 'start_lat',
                            'lng0': 'start_lon',
                            'lat1': 'end_lat',
                            'lng1': 'end_lon'
                        },
                        'isVisible': True,
                        'visConfig': {
                            'opacity': 0.8,
                            'thickness': 'proportion',  # 비율에 따라 굵기 설정
                            'colorRange': {
                                'name': 'Global Warming',
                                'type': 'sequential',
                                'category': 'Uber',
                                'colors': [
                                    '#5A1846',
                                    '#900C3F',
                                    '#C70039',
                                    '#E3611C',
                                    '#F1920E',
                                    '#FFC300'
                                ]
                            },
                            'sizeRange': [1, 10],  # 비율에 따라 굵기 범위 설정
                            'targetColor': None
                        },
                        'textLabel': [
                            {
                                'field': None,
                                'color': [255, 255, 255],
                                'size': 18,
                                'offset': [0, 0],
                                'anchor': 'start',
                                'alignment': 'center'
                            }
                        ]
                    },
                    'visualChannels': {
                        'colorField': None,
                        'colorScale': 'quantile',
                        'sizeField': 'proportion',  # 비율을 사용하여 굵기 설정
                        'sizeScale': 'linear'
                    }
                }
            ],
            'interactionConfig': {
                'tooltip': {
                    'fieldsToShow': {
                        'Arc Data': [
                            'name',
                            'proportion'
                        ]
                    },
                    'enabled': True
                },
                'brush': {
                    'size': 0.5,
                    'enabled': True
                }
            },
            'layerBlending': 'normal',
            'splitMaps': [],
            'animationConfig': {
                'currentTime': None,
                'speed': 1
            }
        },
        'mapState': {
            'bearing': 0,
            'dragRotate': True,
            'latitude': 36.7890,
            'longitude': 127.0010,
            'pitch': 0,
            'zoom': 6
        },
        'mapStyle': {
            'styleType': 'dark',
            'topLayerGroups': {},
            'visibleLayerGroups': {
                'label': True,
                'road': True,
                'border': False,
                'building': True,
                'water': True,
                'land': True
            },
            'threeDBuildingColor': [9.665468314072013, 17.18305478057247, 31.1442867897876],
            'mapStyles': {}
        }
    }
}

# 구성 적용
map_1.config = config

# 지도 출력
map_1

In [53]:
map_1.save_to_html(file_name='visualization/3. 아산시 진료현황 및 환자이송 현황/아산시 진료현황 arcmap.html')

Map saved to visualization/3. 아산시 진료현황 및 환자이송 현황/아산시 진료현황 arcmap.html!


#### 119구급출동이력

In [206]:
# 격자 ID와 격자 폴리곤 각각을 매핑한 딕셔너리 생성
map_dic = {}
for _, row in grid_map_df.iterrows():
    map_dic[row['properties.gid']] = row['geometry']

# 'geometry' 열 값이 특정 폴리곤 내에 있는지 확인하고 'emd_nm' 값을 변경하는 함수
def update_emd_nm(row, polygons, names):
    for polygon, name in zip(polygons, names):
        if row.geometry.intersects(polygon):
            return name
    return row.emd_nm

# 119구급출동이력 데이터프레임
df_emer119 = pd.read_csv('SBJ_2405_001/28.아산시_119구급출동이력.csv')
df_emer119 = df_emer119[df_emer119['sgg_nm'] == '아산시']
df_emer119['emd_nm'] = df_emer119['emd_nm'].replace('배방면', '배방읍') # 배방면 -> 배방읍 변경됨

# map_dic(격자:폴리곤)에 df_emer119의 gid를 매핑하여 'geometry' column 생성
geometry_lst = []
for _, row in df_emer119.iterrows():
    try:
        geometry_lst.append(map_dic[row['gid']])
    except:
        geometry_lst.append(None)
df_emer119['geometry'] = geometry_lst
df_emer119 = df_emer119[~df_emer119['geometry'].isnull()]
df_emer119 = gpd.GeoDataFrame(df_emer119, geometry='geometry')

# 'geometry' 열 값이 특정 폴리곤 내에 있는지 확인하고 'emd_nm' 값 변경
polygons = asan_gdf['geometry'].tolist()
names = asan_gdf['ADM_NM'].tolist()
df_emer119['emd_nm'] = df_emer119.progress_apply(update_emd_nm, axis=1, polygons=polygons, names=names)

# NaN 값을 0으로 대체
df_emer119 = df_emer119.fillna(0)

# 날짜 컬럼을 정수로 변환
df_emer119['arrv_ymd'] = df_emer119['arrv_ymd'].astype(int)
df_emer119['cntct_ymd'] = df_emer119['cntct_ymd'].astype(int)
df_emer119['hm_ymd'] = df_emer119['hm_ymd'].astype(int)
df_emer119['hpt_arrv_ymd'] = df_emer119['hpt_arrv_ymd'].astype(int)

# 날짜와 시간 컬럼을 문자열로 변환
for col in df_emer119.columns[:-1]:
    df_emer119[col] = df_emer119[col].astype(str)

# 시간 형식을 'HH:MM:SS'로 변환하는 함수
def format_time(time_str):
    if time_str == '0':
        return '00:00:00'
    parts = time_str.split(':')
    if len(parts) == 2:
        return f"{int(parts[0]):02}:{int(parts[1]):02}:00"
    return f"{int(parts[0]):02}:{int(parts[1]):02}:{int(parts[2]):02}"

# 날짜와 시간 컬럼을 통합하여 새로운 시간 컬럼 생성 함수
def combine_datetime(date_col, time_col):
    formatted_date = date_col.apply(lambda x: pd.to_datetime(x, format='%Y%m%d', errors='coerce'))
    formatted_time = time_col.apply(lambda x: format_time(x))
    combined = formatted_date.astype(str) + ' ' + formatted_time
    combined = combined.replace('NaT 00:00:00', np.nan)
    return pd.to_datetime(combined, errors='coerce')

df_emer119['신고시각'] = combine_datetime(df_emer119['rpt_ymd'], df_emer119['rpt_tm'])
df_emer119['출동시각'] = combine_datetime(df_emer119['mv_ymd'], df_emer119['mv_tm'])
df_emer119['도착시각'] = combine_datetime(df_emer119['arrv_ymd'], df_emer119['arrv_tm'])
df_emer119['접촉시각'] = combine_datetime(df_emer119['cntct_ymd'], df_emer119['cntct_tm'])
df_emer119['귀소시각'] = combine_datetime(df_emer119['hm_ymd'], df_emer119['hm_tm'])
df_emer119['병원도착시각'] = combine_datetime(df_emer119['hpt_arrv_ymd'], df_emer119['hpt_arrv_tm'])
df_emer119['신고시각'] = df_emer119['신고시각'].astype(str)
df_emer119['출동시각'] = df_emer119['출동시각'].astype(str)
df_emer119['도착시각'] = df_emer119['도착시각'].astype(str)
df_emer119['접촉시각'] = df_emer119['접촉시각'].astype(str)
df_emer119['귀소시각'] = df_emer119['귀소시각'].astype(str)
df_emer119['병원도착시각'] = df_emer119['병원도착시각'].astype(str)

# 불필요한 컬럼 제거
df_emer119.drop(columns=['rpt_ymd', 'rpt_tm', 'mv_ymd', 'mv_tm', 'arrv_ymd', 'arrv_tm', 'cntct_ymd', 'cntct_tm', 'hm_ymd', 'hm_tm', 'hpt_arrv_ymd', 'hpt_arrv_tm'], inplace=True)

df_emer119

100%|██████████| 53598/53598 [00:24<00:00, 2164.98it/s]


,frstn_nm,sfcntr_nm,rslt_gbn,ptnt_type,smptm1,sgg_nm,emd_nm,accdnt_plc,ptnt_age,ptnt_gndr,trnsfr_gbn,hpt_nm,gid,geometry,신고시각,출동시각,도착시각,접촉시각,귀소시각,병원도착시각
0,아산소방서,아산119구조구급센터,취소,질병외,0,아산시,온양6동,0,미확인,미확인,미이송,0,다바569639,"POLYGON ((127.01704 36.77224, 127.01703 36.773...",2021-01-01 00:47:00,2021-01-01 00:53:00,NaT,NaT,2021-01-01 00:55:00,NaT
1,아산소방서,아산119구조구급센터,정상,질병,두통,아산시,온양1동,집,60대,여,이송,아산충무병원,다바552660,"POLYGON ((126.99787 36.79109, 126.99786 36.791...",2021-01-01 10:08:00,2021-01-01 10:10:00,2021-01-01 10:16:00,2021-01-01 10:20:00,2021-01-01 10:55:00,2021-01-01 10:40:00
2,아산소방서,아산119구조구급센터,정상,질병,기타,아산시,온양5동,기타(생활치료센터),20대,여,이송,서울특별시서남병원,다바543609,"POLYGON ((126.98808 36.74507, 126.98808 36.745...",2021-01-01 11:26:00,2021-01-01 12:15:00,2021-01-01 12:28:00,2021-01-01 12:31:00,2021-01-01 16:10:00,2021-01-01 14:10:00
3,아산소방서,아산119구조구급센터,정상,질병외,0,아산시,온양4동,상업시설,미확인,미확인,소방활동,0,다바533669,"POLYGON ((126.97652 36.79911, 126.97651 36.800...",2021-01-01 11:41:00,2021-01-01 11:43:00,2021-01-01 11:50:00,NaT,2021-01-01 12:30:00,NaT
4,신창119안전센터,선장119지역대,정상,질병,기타,아산시,온양5동,기타(생활치료센터),20대,남,이송,충청남도 천안의료원,다바543609,"POLYGON ((126.98808 36.74507, 126.98808 36.745...",2021-01-01 12:11:00,2021-01-01 12:16:00,2021-01-01 12:31:00,2021-01-01 12:35:00,2021-01-01 14:50:00,2021-01-01 13:10:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57773,아산소방서,탕정119안전센터,정상,질병외,0,아산시,배방읍,집,미확인,미확인,미이송,0,다바597638,"POLYGON ((127.04842 36.77146, 127.04841 36.772...",2023-12-31 21:39:00,2023-12-31 21:43:00,2023-12-31 21:55:00,NaT,2023-12-31 22:17:00,NaT
57777,신창119안전센터,선장119지역대,정상,질병외,기타,아산시,선장면,집,70대,여,이송,아산충무병원,다바443671,"POLYGON ((126.87562 36.80042, 126.87561 36.801...",2023-12-31 23:11:00,2023-12-31 23:13:00,2023-12-31 23:17:00,2023-12-31 23:19:00,2024-01-01 00:32:00,2023-12-31 23:50:00
57784,아산소방서,아산119구조구급센터,정상,질병,흉통,아산시,신창면,집,60대,남,이송,순천향대학교부속 천안병원,다바512657,"POLYGON ((126.95305 36.78819, 126.95305 36.789...",2023-12-31 06:58:00,2023-12-31 06:59:00,2023-12-31 07:10:00,2023-12-31 07:10:00,2023-12-31 08:12:00,2023-12-31 07:33:00
57786,아산소방서,배방119안전센터,취소,질병외,0,아산시,온양5동,0,미확인,미확인,미이송,0,다바550635,"POLYGON ((126.99577 36.76854, 126.99577 36.769...",2023-12-31 08:27:00,2023-12-31 08:27:00,NaT,NaT,2023-12-31 08:28:00,NaT


#### 이송환자 연령대 시각화

In [ ]:
# 'ptnt_age' 값의 개수 세기
col_lst = ['10세미만', '10대', '20대', '30대', '40대', '50대', '60대', '70대', '80대', '90세이상']
age_counts = df_emer119['ptnt_age'].value_counts().reindex(col_lst, fill_value=0).reset_index()
age_counts.columns = ['ptnt_age', 'count']

# 시각화
fig = px.bar(
    age_counts,
    x='ptnt_age',
    y='count',
    labels={'ptnt_age': 'Age Group', 'count': 'Count'},
    category_orders={'ptnt_age': col_lst},
    color='count',  # count 열에 따라 색상이 달라짐
    color_continuous_scale='Plasma_r',  # Plasma 색상 맵 사용
)
fig.update_layout(height=600, width=1000)
fig.show()

#### 행정동별 이송환자 시각화

In [ ]:
# 'emd_nm' 값의 개수 세기
emd_nm_counts = df_emer119['emd_nm'].value_counts().reset_index()
emd_nm_counts.columns = ['emd_nm', 'count']

# 시각화
fig = px.bar(
    emd_nm_counts,
    x='emd_nm',
    y='count',
    color='count',  # count 값에 따라 색상이 달라짐
    color_continuous_scale='Plasma_r',  # Plasma 색상 맵 사용
    labels={'emd_nm': 'Region', 'count': 'Count'},
)

fig.show()

In [ ]:
from branca.colormap import linear
# 행정동별 응급출동이력 dict 생성 및 mapping
hjd_119_dic = {key:value for key, value in zip(emd_nm_counts['emd_nm'], emd_nm_counts['count'])}
asan_gdf['119'] = asan_gdf['ADM_NM'].map(hjd_119_dic)

# 색상 맵 생성
colormap = linear.RdPu_07.scale(asan_gdf['119'].min(), asan_gdf['119'].max())

# map 생성
m = folium.Map(location=Asan, tiles='cartodbpositron', zoom_start=11)

# 각 점을 Folium에 추가
for _, row in asan_gdf.iterrows():
    color = colormap(row['119'])
    folium.GeoJson(
        row['geometry'],
        style_function=lambda feature, color=color: {'fillColor': color, 'color': 'black', 'weight': 1}
    ).add_to(m)

# 색상 맵 추가
colormap.add_to(m)

# 맵 출력
m

In [179]:
m.save("visualization/3. 아산시 진료현황 및 환자이송 현황/아산시 행정동별 이송환자 건수 시각화.html")

#### 응급출동 이송분류(KeplerGL 시각화)

In [ ]:
# 격자별 응급출동건수 df 생성
df_emer119_gridcount = pd.DataFrame(df_emer119['geometry'].value_counts()).reset_index()
df_emer119_gridcount = gpd.GeoDataFrame(df_emer119_gridcount, crs='EPSG:4326')

# 맵 객체 생성 및 데이터 로드
emer119_map = KeplerGl(height=800)
emer119_map.add_data(data=asan_gdf, name="아산시 데이터")
emer119_map.add_data(data=df_emer119, name= "응급출동 데이터")
emer119_map.add_data(data=df_emer119_gridcount, name= "격자별 응급출동건수 데이터")
emer119_map.add_data(data=roadsystem_df, name= "도로망 데이터")
# 맵 출력 및 상세설정
emer119_map

In [222]:
emer119_map.save_to_html(file_name="visualization/3. 아산시 진료현황 및 환자이송 현황/응급출동 시각화 keplerGL.html")

Map saved to visualization/3. 아산시 진료현황 및 환자이송 현황/응급출동 시각화 keplerGL.html!
